## CMB3

### Importation of the modules

In [ ]:
import sys
print("Using Python {}".format(sys.version_info[0]))

from pylab import *
import numpy as np
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(17,10))
rc('font',size=12)
rc('text',usetex=False)
rcParams['image.cmap'] = 'jet'
!pip install emcee
!pip install iminuit
!rm -rf cosmolib.py*
!wget --no-check-certificate https://raw.githubusercontent.com/jchamilton75/CS-ST4-Cosmo-2020/master/cosmolib.py
import cosmolib as cs

!pip install healpy
import healpy as hp

#### Reading file from Google Drive
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using Python 3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.4/444.4 kB 4.9 MB/s eta 0:00:00
--2025-06-02 12:09:46--  https://raw.githubusercontent.com/jchamilton75/CS-ST4-Cosmo-2020/master/cosmolib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16798 (16K) [text/plain]
Saving to: ‘cosmolib.py’

cosmolib.py         100%[===================>]  16.40K  --.-KB/s    in 0.001s  

2025-06-02 12:09:46 (13.4 MB/s) - ‘cosmolib.py’ saved [16798/16798]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 41.6 MB/s eta 0:00:00


MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install git+https://github.com/marius311/pypico
import pypico

download = drive.CreateFile({'id': '17DWeh3xPec2iOXrQxNN2aKNr0uxuoahb'})
download.GetContentFile('jcset_py3.dat')
!ls -lrt

pico = pypico.load_pico("jcset_py3.dat")


  Cloning https://github.com/marius311/pypico to /tmp/pip-req-build-nxaecdyw
  Running command git clone --filter=blob:none --quiet https://github.com/marius311/pypico /tmp/pip-req-build-nxaecdyw
  Resolved https://github.com/marius311/pypico to commit a06ced4228d451ebd869d3eb4ea8ed69ee25a1e5
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


ModuleNotFoundError: No module named 'pypico'

### Correction of the error bars

Analytical sample variance

We know that we can express $C_l$ in function of the Spherical Harmonics Transform coefficients of the map $a_{\ell m}$ :  
$$C_l = Var(a_{\ell m})$$
One important thing to note is that $a_{\ell m}$ distribution is centered Gaussian.

We will calculate an empirical $C_l$ knowing that for each value of $l$, there are $2l+1$ values of $m$, which leads to :    
$$C_l = \frac{1}{2\ell+1} \sum_{m=-\ell}^{\ell} \left| a_{\ell m}\right|^2$$

For a given $l$, let's note $\sigma^2$ the variance of $a_{\ell m}$. Supposing for $\forall m \ne n \in [-l, l]$   $a_{\ell m}$ independant of $a_{\ell n}  $, according to Cochran theorem, $ \sum_{m=-\ell}^{\ell} \left| a_{\ell m}\right|^2$ follow a $\chi^2$ distribution with $n$ degree of freedom. So $$Var \left( \sum_{m=-\ell}^{\ell} \left| a_{\ell m}\right|^2 \right) = 2n \sigma^4 = 2(2l+1) Var(a_{\ell m})^2 = 2(2l+1) C_l^2$$.

Therefore we have:     
$$Var(C_l) =  Var \left( \sum_{m=-\ell}^{\ell} \left| a_{\ell m}\right|^2 \right) = \frac{2}{2\ell+1} C_l^2 $$

Knowing that $D_l = \frac{l(l+1) C_l}{2 \pi}$ we have :
$$Var(D_l) =  \frac{l^2(l+1)^2}{4 \pi^2} Var(C_l) = \frac{2}{2\ell+1} D_l^2 $$

There are two more informations to take into account.
the spectrum is binned with $\Delta l=10$ and the CMB is made over a sky fraction of $f = 10\%$. So we finally have   :    
$$Var(D_l) = \frac{2}{(2\ell+1) \Delta l f} D_l^2 $$

We can note that in our case $\Delta l f = 1$


In [ ]:
deltal = 10
f = 0.1
var_dl = (2/((2*ellvalues+1)*deltal*f))*dlexp**2
errors_bar =  np.sqrt(dlnoise**2+ var_dl )

xlabel('$\ell$')
ylabel('$\ell(\ell+1)C_\ell / 2\pi$')
errorbar(ellvalues, dlexp, yerr=errors_bar, fmt='ro')
title("$D_l$ in function of $l$ including sample variance ")

NameError: name 'ellvalues' is not defined

### Best fit of the curve

Now we will now search for the cosmic parameters $h$, $\Omega_m$ and $\Omega_\Lambda$ which fit with this graph. To do so, we need to install PICO, which allows us to simulate a CMB with 8 chosen cosmic parameters. These are stored below in the dictionnary "parpico".

For our project, 5 out of the 8 inputs of PICO are constraints with the following values:    
- ombh2: $\Omega_b h^2$ the baryonic matter density multiplied by the Hubble constant divided by 100
- As: Amplitude of the primordial spectrum for scalars
- ns: $n_s$ the spectral index of the primordial spectrum for scalar modes
- tau: the optical depth of reionization: probability of transmission  $\exp(-\tau)$
- pivot_scalar: the scale at which the scalar power spectrum has a turnaround, kept at PICO default value

We write a function representing the model to be feated

In [ ]:
planck_params = pico.example_inputs()
data = pico.get(**planck_params)
planck_params['ombh2'] = 0.02079
planck_params['As'] = np.exp(3.094)/1e10
planck_params['ns'] = 0.9645
planck_params['tau'] = 0.079
ell = np.arange(len(data['dl_TT']) - 2) + 2  # Define ell values (2 to len(dl_TT))
dl_TT = data['dl_TT']

def nearest_neighbor_index(ell, x): #Finds the index of the element in ell that is closest to the value x.

  # Calculate absolute differences between x and each ell value
  index_list=[]
  for i in range (len(x)):

    differences = np.abs(ell - x[i])
    index_list.append(np.argmin(differences))

  # Find the index of the minimum difference
  return index_list

def model_function(x, param):
  """
  Calculates the CMB temperature power spectrum (dl_TT) at a given ell value
  using the provided cosmological parameters and the pico library.

  Args:
      x (float): The ell value for which to calculate dl_TT.
      param : (h, om, ol)

  Returns:
      float: The dl_TT value at the given ell value.
  """

  # Create a copy of the example parameters to avoid modifying the original
  (h, om, ol) = param
  try :
    inputs = pico.example_inputs()
    inputs['H0'] = 100*h
    inputs['omch2'] = om*h**2-planck_params['ombh2']
    inputs['omk'] = 1 - (om+ol)

    # Get dl_TT data using the modified parameters
    data = pico.get(**inputs)
  except Exception as e: #Point is outside of PICO interpolation region.
      #print(e)
      return 10**9 #we return a large value so that the chi2 is highly penalised
  # Find the index of the nearest ell value in the data
  nearest_index = nearest_neighbor_index(ell, x)

  # Return the dl_TT value at the nearest index
  return data['dl_TT'][nearest_index]


Even though our likelyhood is not gaussian, we can say that it is assymptically gaussian thanks to the Central Limit Theorem and thus we can write :
$$
\begin{eqnarray}
A(\vec{d} \mid \vec{\theta}) &=& -2\log\left(\cal{L(\vec{d} \mid \vec{\theta})}\right)\\
&=& -2\log\left( \frac{1}{\prod_{i=0}^{N-1}\sqrt{2\pi\sigma_i^2}}\right) +\sum_{i=0}^{N-1}\frac{\left(y_i-f(x_i,\vec{\theta})\right)^2}{\sigma_i^2}
\end{eqnarray}$$

The first terms does not depend on the parameters $\vec{\theta}$ and therefore does not play any role in the minimization. The second term is exactly the $\chi^2$.
So we can use the $\chi^2$ as an approximation by using the Central Limit Theorem.
$$\chi^2(\vec{d} \mid \vec{\theta}) = \sum_{i=0}^{N-1}\frac{\left(y_i-f(x_i,\vec{\theta})\right)^2}{\sigma_i^2}$$

In [ ]:
def chi2_cost(params, Data_l, Data_Dl, errors_bar, model_function):
  """Calculates chi-squared cost function with error bars for data fitting.

  Args:
      params (array-like): Array containing the parameters of the model.
      Data_l (array-like): Array of independent data points (x-values).
      Data_Dl (array-like): Array of dependent data points (y-values).
      errors_bar (array-like): Array of error bars for the y-data points.
      model_function (function): Function representing the model to be fitted.

  Returns:
      float: The chi-squared cost value.
  """

  return sum(((Data_Dl - model_function(Data_l, params)) / errors_bar)**2)

There are severale methods to minimize the cost we will try all of them to compare them.


#### With Minuit

First we use the Minuit Librairy

In [ ]:

# paramètres initiaux et des bornes
h=planck_params['H0']/100
om=(planck_params['omch2']+planck_params['ombh2'])/h**2
ol=1-(planck_params['omk']+om)
p0 = np.array([h ,om ,ol ])
print("Initial parameters:", p0)
BOUNDS = [(0.3,0.6), (0,1), (0,1)]

In [ ]:
!pip install iminuit

import numpy as np
from iminuit import Minuit

Data_l, Data_Dl = ellvalues, dlexp

# minimisation avec iminuit
def minuit_chi2(param):
    return chi2_cost(param, Data_l, Data_Dl, errors_bar, model_function)

# on crée un objet Minuit avec les paramètres initiaux et les bornes
m = Minuit(minuit_chi2, p0)
m.limits = BOUNDS

#minimisation
m.migrad()

# Obtenir les paramètres minimisés
minimized_params = m.values

print("Minimized parameters:", minimized_params)


#### With Scipy

We minimize using scipy.optimize.minimize :

In [ ]:
from scipy.optimize import minimize
# Minimize the chi-squared cost function
result = minimize(chi2_cost, p0, args=(Data_l, Data_Dl, errors_bar, model_function),
method='Nelder-Mead', bounds=BOUNDS)

# Print the minimized parameters and chi-squared value
print("Minimized parameters:", result.x)
print("Chi-squared at minimum:", chi2_cost(result.x, Data_l, Data_Dl, errors_bar, model_function))


#### Plot of the best curve

In [ ]:
inputs = pico.example_inputs()
h = result.x[0]
om = result.x[1]
ol = result.x[2]
inputs['H0'] = 100*h
inputs['omch2'] = om*h**2-planck_params['ombh2']
inputs['omk'] = 1 - (om+ol)
dl_TT = pico.get(**inputs)['dl_TT']
plot(ell, dl_TT[:len(ell)])
xlabel('$\ell$')
ylabel('$\ell(\ell+1)C_\ell / 2\pi$')
xlim(0,700)
ylim(0,10000)
errorbar(ellvalues, dlexp, yerr=errors_bar, fmt='ro')

### Monte-Carlo-Markov-Chain

In [ ]:
data = cs.Data(Data_l, Data_Dl, errors_bar, model_function)

chain = data.run_mcmc(p0, nbmc=3000, allvariables=['h', 'om','ol'],nsigmas=10**-10)

In [ ]:
plot(chain['h'], chain['om'], ',', alpha=0.1)

xlabel('p0')
ylabel('p1')

In [ ]:
im2d = hist2d(chain['h'], chain['om'], bins=100)
mat = np.ma.masked_where(im2d[0]==0,im2d[0]).T
imshow(mat, interpolation='nearest', origin='lower',aspect='auto')
colorbar()
xlabel('h')
ylabel('om')

In [ ]:
sm = 1.5
cs.matrixplot(chain, ['h', 'om','ol'], 'green', sm, labels=['h', r'$\Omega_m$',r'$\Omega_{\Lambda}$'])